# Day 16: Reindeer Maze

### Part 1

It's time again for the Reindeer Olympics! This year, the big event is the Reindeer Maze, where the Reindeer compete for the lowest score.

You and The Historians arrive to search for the Chief right as the event is about to start. It wouldn't hurt to watch a little, right?

The Reindeer start on the Start Tile (marked S) facing East and need to reach the End Tile (marked E). They can move forward one tile at a time (increasing their score by 1 point), but never into a wall (#). They can also rotate clockwise or counterclockwise 90 degrees at a time (increasing their score by 1000 points).

To figure out the best place to sit, you start by grabbing a map (your puzzle input) from a nearby kiosk. For example:

```
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
```

There are many paths through this maze, but taking any of the best paths would incur a score of only 7036. This can be achieved by taking a total of 36 steps forward and turning 90 degrees a total of 7 times:

```
###############
#.......#....E#
#.#.###.#.###^#
#.....#.#...#^#
#.###.#####.#^#
#.#.#.......#^#
#.#.#####.###^#
#..>>>>>>>>v#^#
###^#.#####v#^#
#>>^#.....#v#^#
#^#.#.###.#v#^#
#^....#...#v#^#
#^###.#.#.#v#^#
#S..#.....#>>^#
###############
```

Here's a second example:

```
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
```

In this maze, the best paths cost 11048 points; following one such path would look like this:

```
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#^#
#.#.#.#...#...#^#
#.#.#.#.###.#.#^#
#>>v#.#.#.....#^#
#^#v#.#.#.#####^#
#^#v..#.#.#>>>>^#
#^#v#####.#^###.#
#^#v#..>>>>^#...#
#^#v###^#####.###
#^#v#>>^#.....#.#
#^#v#^#####.###.#
#^#v#^........#.#
#^#v#^#########.#
#S#>>^..........#
#################
```

Note that the path shown above includes one 90 degree turn as the very first move, rotating the Reindeer from facing East to facing North.

Analyze your map carefully. What is the lowest score a Reindeer could possibly get?




In [16]:
import com.psyndicate.aoc.*
import java.util.PriorityQueue
import kotlin.test.assertEquals

val testInput =
    """
    ###############
    #.......#....E#
    #.#.###.#.###.#
    #.....#.#...#.#
    #.###.#####.#.#
    #.#.#.......#.#
    #.#.#####.###.#
    #...........#.#
    ###.#.#####.#.#
    #...#.....#.#.#
    #.#.#.###.#.#.#
    #.....#...#.#.#
    #.###.#.#.#.#.#
    #S..#.....#...#
    ###############
    """.trimIndent()

val testInput2 =
    """
    #################
    #...#...#...#..E#
    #.#.#.#.#.#.#.#.#
    #.#.#.#...#...#.#
    #.#.#.#.###.#.#.#
    #...#.#.#.....#.#
    #.#.#.#.#.#####.#
    #.#...#.#.#.....#
    #.#.#####.#.###.#
    #.#.#.......#...#
    #.#.###.#####.###
    #.#.#...#.....#.#
    #.#.#.#####.###.#
    #.#.#.........#.#
    #.#.#.#########.#
    #S#.............#
    #################
    """.trimIndent()

data class Path(val path: List<Coord>, val heading: Direction, val points: Int = 0)

fun Bitmap<Char>.search(
    position: Coord,
    heading: Direction,
    goal: Coord
): Pair<Int, Int> {
    var minScore = Int.MAX_VALUE
    val bestSeats = mutableSetOf<Coord>()
    val seen = mutableMapOf<Pair<Coord, Direction>, Int>()
    val queue = PriorityQueue(compareBy(Path::points))
    queue.add(Path(listOf(position), heading))
    while (queue.isNotEmpty()) {
        queue.addAll(
            queue.poll()
                .takeIf { (path, heading, points) ->
                    val location = path.last() to heading
                    val bestScore = seen.getOrDefault(location, Int.MAX_VALUE)
                    if (points < bestScore) seen[location] = points
                    if (location.first == goal) {
                        if (points > minScore) {
                            // Scores are getting worse
                            return minScore to bestSeats.size
                        }
                        minScore = points
                        bestSeats += path
                    }
                    points <= bestScore
                }
                ?.let {
                    listOf(
                        it.heading to 1,                                   // Forward
                        it.heading.rotate90(clockwise = true) to 1001,     // Rotate clockwise
                        it.heading.rotate90(clockwise = false) to 1001,    // Rotate counter-clockwise
                    ).map { (nh, cost) -> Path(it.path + (it.path.last() + nh), nh, it.points + cost) }
                        .filter { this@search[it.path.last()] != '#' }
                }
                ?: emptyList()
        )

    }
    error("No path found")
}

fun calculatePart1(input: String): Int {
    val map = bitmapFromString(input)
    val start = map.coords().firstOrNull { map[it] == 'S' } ?: error("Couldn't find start")
    val end = map.coords().firstOrNull { map[it] == 'E' } ?: error("Couldn't find end")
    return map.search(start, Direction.RIGHT, end).first
}

val testInputPart1Result = calculatePart1(testInput)
DISPLAY("Test Part 1: ${testInputPart1Result}")
assertEquals(7036, testInputPart1Result)

val testInput2Part1Result = calculatePart1(testInput2)
DISPLAY("Test 2 Part 1: ${testInput2Part1Result}")
assertEquals(11048, testInput2Part1Result)

val puzzleInput = readResourceAsText("/day16/input.txt")
val part1Result = calculatePart1(puzzleInput)
DISPLAY("Part 1: ${part1Result}")
assertEquals(133584, part1Result)

Test Part 1: 7036

Test 2 Part 1: 11048

Part 1: 133584

### Part 2

Now that you know what the best paths look like, you can figure out the best spot to sit.

Every non-wall tile (S, ., or E) is equipped with places to sit along the edges of the tile. While determining which of these tiles would be the best spot to sit depends on a whole bunch of factors (how comfortable the seats are, how far away the bathrooms are, whether there's a pillar blocking your view, etc.), the most important factor is whether the tile is on one of the best paths through the maze. If you sit somewhere else, you'd miss all the action!

So, you'll need to determine which tiles are part of any best path through the maze, including the S and E tiles.

In the first example, there are 45 tiles (marked O) that are part of at least one of the various best paths through the maze:

```
###############
#.......#....O#
#.#.###.#.###O#
#.....#.#...#O#
#.###.#####.#O#
#.#.#.......#O#
#.#.#####.###O#
#..OOOOOOOOO#O#
###O#O#####O#O#
#OOO#O....#O#O#
#O#O#O###.#O#O#
#OOOOO#...#O#O#
#O###.#.#.#O#O#
#O..#.....#OOO#
###############
```

In the second example, there are 64 tiles that are part of at least one of the best paths:

```
#################
#...#...#...#..O#
#.#.#.#.#.#.#.#O#
#.#.#.#...#...#O#
#.#.#.#.###.#.#O#
#OOO#.#.#.....#O#
#O#O#.#.#.#####O#
#O#O..#.#.#OOOOO#
#O#O#####.#O###O#
#O#O#..OOOOO#OOO#
#O#O###O#####O###
#O#O#OOO#..OOO#.#
#O#O#O#####O###.#
#O#O#OOOOOOO..#.#
#O#O#O#########.#
#O#OOO..........#
#################
```
Analyze your map further. How many tiles are part of at least one of the best paths through the maze?

In [15]:
fun calculatePart2(input: String): Int {
    val map = bitmapFromString(input)
    val start = map.coords().firstOrNull { map[it] == 'S' } ?: error("Couldn't find start")
    val end = map.coords().firstOrNull { map[it] == 'E' } ?: error("Couldn't find end")
    return map.search(start, Direction.RIGHT, end).second
}

val testInputPart2Result = calculatePart2(testInput)
DISPLAY("Test Part 2: ${testInputPart2Result}")
assertEquals(45, testInputPart2Result)

val testInput2Part2Result = calculatePart2(testInput2)
DISPLAY("Test 2 Part 2: ${testInput2Part2Result}")
assertEquals(64, testInput2Part2Result)

val part2Result = calculatePart2(puzzleInput)
DISPLAY("Part 2: ${part2Result}")
assertEquals(622, part2Result)

Test Part 2: 45

Test 2 Part 2: 64

Part 2: 622